# **MC536 - Trabalho Final**

### **Equipe Coviders**

Fernando dos R. S. Filho - RA: 234471

Renan Hiroki Bastos - RA: 176573

Vinicius Alves Mancine Dantas - RA: 188092

# Tratamentos

In [ ]:
# IMPORTAR ARQUIVOS CSV PARA EUROPA
DROP TABLE IF EXISTS dados_owid;
DROP TABLE IF EXISTS dados_yougov;

CREATE TABLE dados_owid (
    id INTEGER NOT NULL,
	iso_code VARCHAR(8),
	continent VARCHAR(20),
	location VARCHAR(35),
	date VARCHAR(20),
	total_cases DEC(12, 2),
	new_cases DEC(12, 2),
	new_cases_smoothed DEC(12, 2),
	total_deaths DEC(12, 2),
	new_deaths DEC(12, 2),
	new_deaths_smoothed DEC(12, 2),
	total_cases_per_million DEC(12, 2),
	new_cases_per_million DEC(12, 2),
	new_cases_smoothed_per_million DEC(12, 2),
	total_deaths_per_million DEC(12, 2),
	new_deaths_per_million DEC(12, 2),
	new_deaths_smoothed_per_million DEC(12, 2),
	reproduction_rate DEC(12, 2),
	icu_patients DEC(12, 2),
	icu_patients_per_million DEC(12, 2),
	hosp_patients DEC(12, 2),
	hosp_patients_per_million DEC(12, 2),
	weekly_icu_admissions DEC(12, 2),
	weekly_icu_admissions_per_million  DEC(12, 2),
	weekly_hosp_admissions DEC(12, 2),
	weekly_hosp_admissions_per_million DEC(12, 2),
	new_tests DEC(12, 2),
	total_tests DEC(12, 2),
	total_tests_per_thousand DEC(12, 2),
	new_tests_per_thousand DEC(12, 2),
	new_tests_smoothed DEC(12, 2),
	new_tests_smoothed_per_thousand DEC(12, 2),
	positive_rate DEC(12, 2),
	tests_per_case DEC(12, 2),
	tests_units VARCHAR(20),
	total_vaccinations DEC(12, 2),
	people_vaccinated DEC(12, 2),
	people_fully_vaccinated DEC(12, 2),
	total_boosters DEC(12, 2),
	new_vaccinations DEC(12, 2),
	new_vaccinations_smoothed DEC(12, 2),
	total_vaccinations_per_hundred DEC(12, 2),
	people_vaccinated_per_hundred DEC(12, 2),
	people_fully_vaccinated_per_hundred DEC(12, 2),
	total_boosters_per_hundred DEC(12, 2),
	new_vaccinations_smoothed_per_million DEC(12, 2),
	stringency_index DEC(12, 2),
	population DEC(12, 2),
	population_density DEC(12, 2),
	median_age DEC(12, 2),
	aged_65_older DEC(12, 2),
	aged_70_older DEC(12, 2),
	gdp_per_capita DEC(12, 2),
	extreme_poverty DEC(12, 2),
	cardiovasc_death_rate DEC(12, 2),
	diabetes_prevalence DEC(12, 2),
	female_smokers DEC(12, 2),
	male_smokers DEC(12, 2),
	handwashing_facilities DEC(12, 2),
	hospital_beds_per_thousand DEC(12, 2),
	life_expectancy DEC(12, 2),
	human_development_index DEC(12, 2),
	excess_mortality DEC(12, 2),
	PRIMARY KEY(ID)
);

COPY dados_owid
FROM '../data/raw/owid-covid-data.csv'
DELIMITER ','
CSV HEADER;

CREATE TABLE dados_yougov (
	id INTEGER NOT NULL,
	date VARCHAR(10),
	location VARCHAR(20),
	mask_use_percentage DEC(4, 2),
	PRIMARY KEY(ID)
);

COPY dados_yougov
FROM '../data/external/yougov-chart.csv'
DELIMITER ','
CSV HEADER;

In [ ]:
# TRATAR DADOS DA OUR WORLD IN DATA (EUROPA)

DROP VIEW IF EXISTS view_europa;
DROP TABLE IF EXISTS dados_owid_tratados;

CREATE VIEW view_europa AS
SELECT id,
	   location,
	   date,
	   total_cases,	
	   LAG(total_cases) OVER (
		   ORDER BY id) old_total_cases,
	   total_deaths,
	   LAG(total_deaths) OVER (
		   ORDER BY id) old_total_deaths
FROM dados_owid
WHERE (location='Denmark' OR location='Finland' OR location='France' OR location='Germany' OR location='Italy' OR location='Norway' OR location='Spain' OR location='Sweden' OR location='United Kingdom')
		AND (date='2020-02-01' OR date='2020-03-01' OR date='2020-04-01' OR date='2020-05-01' OR date='2020-06-01' OR date='2020-07-01' OR date='2020-08-01' OR date='2020-09-01' OR date='2020-10-01' OR date='2020-11-01' OR date='2020-12-01' OR date='2021-01-01' OR date='2021-02-01' OR date='2021-03-01' OR date='2021-04-01' OR date='2021-05-01' OR date='2021-06-01' OR date='2021-07-01' OR date='2021-08-01');

CREATE TABLE dados_owid_tratados AS
	SELECT id,
	   location,
	   date,
	   total_cases,	
	   total_cases - old_total_cases as new_cases,
	   total_deaths,
	   total_deaths - old_total_deaths as new_deaths
FROM view_europa;

UPDATE dados_owid_tratados
SET new_cases = total_cases
WHERE (date = '2020-02-01' AND new_cases < 0) OR (date = '2020-03-01' AND new_cases < 0);

In [ ]:
# JOIN DAS TABELAS DE NUMEROS DE CASOS E USO DE MÁSCARA PARA A EUROPA

CREATE TABLE europa_final AS
SELECT eu.*, mask.mask_use_percentage 
FROM dados_owid_tratados eu
INNER JOIN dados_yougov mask ON (eu.date = mask.date) AND (eu.location = mask.location);

In [ ]:
# IMPORTAR ARQUIVOS CSV PARA OS EUA

DROP TABLE IF EXISTS dados_nyt;

CREATE TABLE dados_nyt (
	id INTEGER NOT NULL,
	date VARCHAR(10),
	state VARCHAR(40),
	fips INTEGER,
	cases INTEGER,
	deaths INTEGER,
	PRIMARY KEY(ID)
);

COPY dados_nyt
FROM '../data/external/nyt.csv'
DELIMITER ','
CSV HEADER;

DROP TABLE IF EXISTS dados_plosone;

CREATE TABLE dados_plosone (
	StateID INTEGER,
	State VARCHAR(20),
	adh_AUGavg DEC(4, 2),
	adh_JULavg DEC(4, 2),
	adh_JUNEavg DEC(4, 2),
	adh_MAYavg DEC(4, 2),
	adh_APRavg DEC(4, 2),
	Pop_2019 INTEGER,
	Hispanic DEC(4, 2),
	NHBlack DEC(4, 2),
	NHWhite DEC(4, 2),
	Age DEC(4, 2),
	PopDensity DEC(8, 2),
	DistancePolicy VARCHAR(20),
	tot_Cases_Jul INTEGER,
	DistanceNone INTEGER,
	DistanceStrict INTEGER,
	DistCat INTEGER,
	tot_cases_Apr INTEGER,
	tot_cases_May INTEGER,
	tot_cases_Jun INTEGER,
	tot_cases_Aug INTEGER,
	tot_cases_Sep INTEGER,
	newCases_May INTEGER,
	newCases_Jun INTEGER,
	newCases_Jul INTEGER,
	newCases_Aug INTEGER,
	newCases_Sep INTEGER,
	case_rate_May DEC(8, 2),
	case_rate_Jun DEC(8, 2),
	case_rate_Jul DEC(8, 2),
	case_rate_Aug DEC(8, 2),
	case_rate_Sep DEC(8, 2),
	EndStayHome VARCHAR(10),
	NoStayHomePolicy INTEGER,
	NoHomeMay INTEGER,
	MaskPolStart VARCHAR(10),
	MaskJunPol INTEGER,
	MaskJulPol INTEGER,
	MaskAugPol INTEGER,
	MaskSepPol INTEGER,
	MaskPolStop INTEGER,
	tot_cases_Oct INTEGER,
	newCases_Oct INTEGER,
	case_rate_Oct DEC(8, 2),
	AdhSEPavg DEC(8, 2),
	Wk_case_rate_Sep DEC(8, 2),
	Wk_case_rate_Oct DEC(8, 2),
	Wk_case_rate_Aug DEC(8, 2),
	Wk_case_rate_Jul DEC(8, 2),
	Wk_case_rate_Jun DEC(8, 2),
	Wk_case_rate_May DEC(8, 2),
	Wk_MayHI INTEGER,
	Wk_JunHI INTEGER,
	Wk_JulHI INTEGER,
	Wk_AugHI INTEGER,
	Wk_SepHI INTEGER,
	Wk_OctHI INTEGER,
	maskGT75 INTEGER,
	maskSEP75 INTEGER,
	maskJUL75 INTEGER,
	maskAUG75 INTEGER,
	strictJunPol INTEGER,
	strictJulPol INTEGER,
	strictSeplPol INTEGER,
	strictSepPol INTEGER,
	strictAuglPol INTEGER,
	strictAugPol INTEGER,
	PRIMARY KEY(StateID)
);

COPY dados_plosone
FROM '../data/external/plosone.csv'
DELIMITER ','
CSV HEADER;

In [ ]:
# TRATAR DADOS DO NYT (EUA)

DROP VIEW IF EXISTS view_eua;
DROP TABLE IF EXISTS table_eua;

CREATE VIEW view_eua AS
SELECT temp_data.id,
	   temp_data.location,
	   temp_data.date,
	   temp_data.cases as total_cases,
	   LAG(temp_data.cases) OVER (
		   ORDER BY temp_data.location, temp_data.id) AS old_total_cases,
	   temp_data.deaths as total_deaths,
	   LAG(temp_data.deaths) OVER (
		   ORDER BY temp_data.location, temp_data.id) AS old_total_deaths
FROM (SELECT id,
		 state as location,
		 date,
		 cases,
		 deaths
		 FROM dados_nyt
		 ORDER BY state, id) as temp_data
WHERE (date='2020-02-01' OR date='2020-03-01' OR date='2020-04-01' OR date='2020-05-01' OR date='2020-06-01' OR date='2020-07-01' OR date='2020-08-01' OR date='2020-09-01' OR date='2020-10-01' OR date='2020-11-01' OR date='2020-12-01' OR date='2021-01-01' OR date='2021-02-01' OR date='2021-03-01' OR date='2021-04-01' OR date='2021-05-01' OR date='2021-06-01' OR date='2021-07-01' OR date='2021-08-01');

CREATE TABLE table_eua AS
	SELECT id,
	   location,
	   date,
	   total_cases,	
	   total_cases - old_total_cases as new_cases,
	   total_deaths,
	   total_deaths - old_total_deaths as new_deaths
FROM view_eua;

UPDATE table_eua
SET new_cases = total_cases,
	new_deaths = total_deaths
WHERE (date = '2020-02-01' AND new_cases < 0) OR (date = '2020-03-01' AND new_cases < 0) OR (date = '2020-04-01' AND new_cases < 0) OR (date = '2020-05-01' AND new_cases < 0);

SELECT * FROM table_eua;

In [ ]:
# TRATAR DADOS DO PLOS ONE (USO DE MÁSCARA NOS EUA)

DROP TABLE IF EXISTS dados_plosone_tratados;

CREATE TABLE dados_plosone_tratados (
	date VARCHAR(10),
	location VARCHAR(40),
	mask_use_percentage DEC(4, 2)
);

INSERT INTO dados_plosone_tratados (location, date, mask_use_percentage)
SELECT State, '2020-04-01', adh_APRavg
FROM dados_plosone;

INSERT INTO dados_plosone_tratados (location, date, mask_use_percentage)
SELECT State, '2020-05-01', adh_MAYavg
FROM dados_plosone;

INSERT INTO dados_plosone_tratados (location, date, mask_use_percentage)
SELECT State, '2020-06-01', adh_JUNEavg
FROM dados_plosone;

INSERT INTO dados_plosone_tratados (location, date, mask_use_percentage)
SELECT State, '2020-07-01', adh_JULavg
FROM dados_plosone;

INSERT INTO dados_plosone_tratados (location, date, mask_use_percentage)
SELECT State, '2020-08-01', adh_AUGavg
FROM dados_plosone;

INSERT INTO dados_plosone_tratados (location, date, mask_use_percentage)
SELECT State, '2020-09-01', AdhSEPavg
FROM dados_plosone;

In [ ]:
# JOIN DAS TABELAS DE NUMERO DE CASOS E DE USO DE MÁSCARA PARA OS EUA

DROP TABLE IF EXISTS eua_final;

CREATE TABLE eua_final AS
SELECT eua.*, mask.mask_use_percentage 
FROM dados_nyt_tratados eua
INNER JOIN dados_plosone_tratados mask ON ((eua.date = mask.date) AND (eua.location = mask.location));

INSERT INTO eua_final (id, location, date, new_cases, total_cases, new_deaths, total_deaths, mask_use_percentage)
SELECT id, location, '2020-04-01', new_cases, total_cases, new_deaths, total_deaths,  NULL
FROM dados_nyt_tratados;

In [ ]:
# UNINDO AS TABELAS FINAIS DA EUROPA E DOS EUA - processo de construção do datset finalizado


DROP TABLE IF EXISTS tabela_final;

CREATE TABLE tabela_final AS
SELECT * 
FROM europa_final;

INSERT INTO tabela_final(id, location, date, new_cases, total_cases, new_deaths, total_deaths, mask_use_percentage)
    SELECT id, location, date, new_cases, total_cases, new_deaths, total_deaths, mask_use_percentage
    FROM eua_final;

In [ ]:
# CRIANDO A TABELA POPULACAO

CREATE TABLE populacao(
  location VARCHAR(5) NOT NULL,
  population VARCHAR(6) NOT NULL,
  PRIMARY KEY(location)
);
COPY populacao
FROM '../data/processed/populacao.csv'
DELIMITER ','
CSV HEADER;

In [ ]:
# UNINDO DADOS SOBRE POPULACAO A TABELA FINAL

DROP TABLE IF EXISTS tabela_final_final;

CREATE TABLE tabela_final_final AS
SELECT tf.*,
	   tf.new_cases*10^6/po.population AS new_cases_per_million_habitants,
	   tf.new_deaths*10^6/po.population AS new_deaths_per_million_habitants,
	   tf.total_cases*10^6/po.population AS total_cases_per_million_habitants,
	   tf.total_deaths*10^6/po.population AS total_deaths_per_million_habitants
FROM tabela_final tf
INNER JOIN populacoes po ON (tf.location = po.location);

In [ ]:
# CRIANDO O DATASET FINALIZADO

CREATE TABLE casos (
	id INTEGER NOT NULL,
	location VARCHAR(20),
	date DATE,
  total_cases INTEGER,
  new_cases INTEGER,
  total_deaths INTEGER,
  new_deaths INTEGER,
	mask_use_percentage DEC(4, 2),
  new_cases_per_million_habitants DEC(4, 2),
  new_deaths_per_million_habitants DEC(4, 2),
  total_cases_per_million_habitants DEC(4, 2),
  total_deaths_per_million_habitants DEC(4, 2),
  population INTEGER,
	PRIMARY KEY(id),
  FOREIGN KEY(location)
    REFERENCES populacao(location)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
);

COPY casos
FROM '../data/processed/casos.csv'
DELIMITER ','
CSV HEADER;

# Perguntas e análises

Com este dataset é possível responder às seguintes perguntas de análise com suas respectivas queries:

### Os locais com maior número de casos são também os lugares com menor índice de uso de máscaras?

In [ ]:
SELECT sum(new_cases_per_million_habitants),
       avg(new_cases_per_million_habitants)
FROM casos,
WHERE mask_use_percentage < 25;

SELECT sum(new_cases_per_million_habitants),
       avg(new_cases_per_million_habitants)
FROM casos,
WHERE mask_use_percentage >= 25 AND mask_use_percentage < 50;

SELECT sum(new_cases_per_million_habitants),
       avg(new_cases_per_million_habitants)
FROM casos,
WHERE mask_use_percentage >= 50 AND mask_use_percentage < 75; 

SELECT sum(new_cases_per_million_habitants),
       avg(new_cases_per_million_habitants)
FROM casos,
WHERE mask_use_percentage >= 75;

### Quais são os locais com maior número de casos por índice de uso de máscaras?

In [ ]:
SELECT location,
	   date,
	   mask_use_percentage,
	   new_cases_per_million_habitants,
	   new_cases_per_million_habitants/mask_use_percentage as case_mask_use_rate
FROM casos
WHERE mask_use_percentage > 0 AND new_cases > 0
ORDER BY case_mask_rate DESC
LIMIT 20;

### Há algum indício de que a frequência de uso de máscara influência na taxa de mortalidade?

In [ ]:
SELECT  location,
        date,
        mask_use_percentage,
        new_deaths*100/new_cases as monthly_death_rate,
        total_deaths*100/total_cases as overall_death_rate
FROM tabela_final_final
WHERE new_cases != 0 AND total_cases != 0 AND mask_use_percentage IS NOT NULL
ORDER BY mask_use_percentage DESC;

SELECT avg(new_deaths*100/new_cases) as avarage_death_rate
FROM casos
WHERE mask_use_percentage < 25;

SELECT avg(new_deaths*100/new_cases) as avarage_death_rate
FROM casos
WHERE mask_use_percentage >= 25 AND mask_use_percentage < 50;

SELECT avg(new_deaths*100/new_cases) as avarage_death_rate
FROM casos
WHERE mask_use_percentage >= 50 AND mask_use_percentage < 75; 

SELECT avg(new_deaths*100/new_cases) as avarage_death_rate
FROM casos
WHERE mask_use_percentage >= 75;

### Locais na mesma faixa de porcentagem de uso de máscara possuem taxas de infecções parecidas?

In [ ]:
SELECT location,
       new_cases_per_million_habitants
FROM casos,
WHERE mask_use_percentage < 10;

SELECT location,
       new_cases_per_million_habitants
FROM casos,
WHERE mask_use_percentage >= 10 AND mask_user_percentage < 20;

SELECT location,
       new_cases_per_million_habitants
FROM casos,
WHERE mask_use_percentage >= 20 AND mask_user_percentage < 30;

SELECT location,
       new_cases_per_million_habitants
FROM casos,
WHERE mask_use_percentage >= 30 AND mask_user_percentage < 40;

SELECT location,
       new_cases_per_million_habitants
FROM casos,
WHERE mask_use_percentage >= 40 AND mask_user_percentage < 50;

SELECT location,
       new_cases_per_million_habitants
FROM casos,
WHERE mask_use_percentage >= 50 AND mask_user_percentage < 60;

SELECT location,
       new_cases_per_million_habitants
FROM casos,
WHERE mask_use_percentage >= 60 AND mask_user_percentage < 70;
SELECT location,
       new_cases_per_million_habitants
FROM casos,
WHERE mask_use_percentage >= 70 AND mask_user_percentage < 80;

SELECT location,
       new_cases_per_million_habitants
FROM casos,
WHERE mask_use_percentage >= 80 AND mask_user_percentage < 90;

SELECT location,
       new_cases_per_million_habitants
FROM casos,
WHERE mask_use_percentage >= 90;
